In [2]:
import psycopg2

connection = psycopg2.connect(host="localhost", user="root", port=5432, database="W9sV6cL2dX", password="E5rG7tY3fH")
connection.autocommit = True
cursor = connection.cursor()

In [3]:
import dataset
params = { "host": "localhost", "user": "root", "port": 5432, "database": "W9sV6cL2dX", "password": "E5rG7tY3fH" }
text_ds = dataset.TextDataSet(params)
print(text_ds[43])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


example sentence demonstrating text preprocessing .
بسم الله الرحمن الرحيم. مرحبًا بك في هذا الدرس الحزين القصير حول التفاح والكلب والغيتار.

في بستانٍ جميل عاش تفاحٌ صغير، كان قلبه مملوءًا بالأمل والجمال. كان ينبض بالحياة والألوان، يمتلك مذاقًا حلوًا وعبقًا يغمر المكان. ولكن، كان هناك الكثير من النباتات الأخرى حوله، تعطيه الشعور بالقليل من العزلة. 

في نفس الوقت، كان هناك كلبٌ وفيٌ يعيش في بيت صغير قرب البستان. كان صديقًا مخلصًا لأهل البيت، يقدم لهم الحب والولاء. ومع ذلك، لم يكن له رفيقٌ يلعب معه أو يشاركه لحظات الفرح والمرح.

ثم كان هناك غيتارٌ جميل، يرقص على أوتاره بأنغام الحزن والميلانكوليا. يمكنه نقل الأحاسيس والمشاعر بشكل لا يوصف، ولكن لم يكن هناك من يستمع إلى موسيقاه أو يشعر بحنينه.

في يومٍ من الأيام، التقى هؤلاء الثلاثة: التفاح، والكلب، والغيتار. التفاح والكلب أحبوا بعضهم البعض وأصبحوا أصدقاء حقيقيين. عانى التفاح من انعزاله، والكلب من الوحدة، فقررا أن يسافرا سويًا بهدف العثور على شخصٍ يستمع ويقدر الأشياء الجميلة.

سار الصديقان لمسافات طويلة، حتى وصلا إلى مكان مزدحم بالناس. است

In [4]:
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2') #using a relatively smaller size model from the api

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from torch.utils.data import DataLoader

batch_size = 32
dataloader = DataLoader(text_ds, batch_size=batch_size, shuffle=False)

In [19]:
model.eval()

sentence_embeddings = []

sample_batch = next(iter(dataloader))
batches = [sample_batch[0], sample_batch[1]]
with torch.no_grad():
    for batch in batches:
        # Batch contains sentences
        # Convert sentences to list if not already in list format
        if isinstance(batch, torch.Tensor):
            batch = batch.tolist()

        # Encode the sentences in the current batch
        embeddings = model.encode(batch, convert_to_tensor=True)
        sentence_embeddings.append(embeddings)

# Concatenate the embeddings from all batches into a single tensor
sentence_embeddings = torch.cat(sentence_embeddings, dim=0)
print(sentence_embeddings)

tensor([-2.0455e-02,  6.7379e-02, -2.3074e-02,  9.1365e-03, -4.1385e-04,
        -2.3259e-02, -6.5432e-03, -5.3029e-02, -4.0707e-02,  1.0945e-02,
         9.4078e-02,  1.8664e-02,  5.0420e-02,  7.8384e-02,  3.8498e-02,
        -1.6045e-02, -3.0188e-02,  1.5984e-02, -2.1809e-02,  6.0038e-03,
         4.9442e-04, -2.5956e-02,  6.0988e-02, -1.1497e-02, -3.6571e-02,
         3.7859e-02, -4.6173e-02,  1.1374e-02, -5.0658e-02, -6.6496e-03,
         1.5871e-02, -3.8937e-02,  3.2480e-03, -8.0666e-02, -5.9401e-02,
         8.1491e-02,  4.6910e-02,  4.4707e-02,  3.5244e-02, -6.6331e-03,
         8.3570e-02, -1.6245e-02,  2.8714e-02, -6.2166e-02, -5.6266e-02,
        -4.5995e-02, -3.9277e-02, -4.0103e-02,  5.9454e-04, -8.9283e-02,
        -2.6406e-02, -7.4847e-03,  3.1943e-02, -1.9263e-02,  2.9358e-02,
        -5.2222e-02, -4.9546e-02,  7.4767e-03,  6.4746e-02,  5.6178e-02,
        -7.4099e-02,  1.4518e-01, -4.6741e-03, -2.6018e-02,  6.1169e-02,
        -1.5379e-02, -1.2255e-02, -5.0244e-02,  2.4

In [20]:
import faiss

sentence_embeddings_np = sentence_embeddings.view(-1, sentence_embeddings.shape[-1]).numpy()

# Create an IndexFlatL2 index
index = faiss.IndexFlatL2(sentence_embeddings_np.shape[1])  # Embedding dimension (384 in your case)

# Add sentence embeddings to the index
index.add(sentence_embeddings_np)

In [23]:
# Define the query sentence
query_sentence = "This is some broccoli haha."

# Encode the query sentence
query_embedding = model.encode(query_sentence, convert_to_tensor=True)

# Convert the query embedding to a numpy array
query_embedding_np = query_embedding.numpy()

# Perform the similarity search
k = 5  # Number of similar sentences to retrieve
_, indices = index.search(query_embedding_np, k)

# Get the similar sentences from your dataset
similar_sentences = [text_ds[i] for i in indices[0]]

print("Query sentence:", query_sentence)
print("Most similar sentences:", similar_sentences)

ValueError: not enough values to unpack (expected 2, got 1)